In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
import awkward as ak
from dask.distributed import Client
#plotting
import matplotlib.pyplot as plt
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from sidm.tools import ffschema, sidm_processor, scaleout
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)

<module 'sidm.tools.sidm_processor' from '/home/cms-jovyan/SIDM/sidm/test_notebooks/../../sidm/tools/sidm_processor.py'>

In [2]:
from dask.distributed import Client

client = Client("tls://bryan-2ecardwell-40cern-2ech.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
client

/opt/conda/lib/python3.8/site-packages/distributed/client.py:1288: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| python  | 3.8.16.final.0 | 3.8.16.final.0 | 3.8.15.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://192.168.41.108:8786' processes=1 threads=2, memory=7.00 GiB>

In [11]:
fileset = {
    "2Mu2E_100GeV_1p2GeV_9p6mm": [
        "root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_1.root",
    ],
}

runner = processor.Runner(
    #executor=processor.IterativeExecutor(),
    executor=processor.FuturesExecutor(),
    #executor=processor.DaskExecutor(client=client),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "2mu2e",
    ],
    ["base"]
)

# test if processor is serializable
import coffea.util as coffea_util
coffea_util.save(p, "processor.coffea")
print(coffea_util.load("processor.coffea"))

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)

Output()

In [15]:
out = output["out"]
print(out["2Mu2)

{'2Mu2E_100GeV_1p2GeV_9p6mm': {'cutflow': {'2mu2e': <sidm.tools.cutflow.Cutflow object at 0x7fb9454ebfd0>}, 'hists': {'pv_n': Hist(
  StrCategory(['2mu2e'], name='channel'),
  Regular(50, 0, 100, name='pv_n'),
  storage=Weight()) # Sum: WeightedSum(value=1913.48, variance=3022.76), 'pv_ndof': Hist(
  StrCategory(['2mu2e'], name='channel'),
  Regular(25, 0, 100, name='pv_ndof'),
  storage=Weight()) # Sum: WeightedSum(value=46181.4, variance=74613.3) (WeightedSum(value=49850, variance=80903.1) with flow), 'pv_z': Hist(
  StrCategory(['2mu2e'], name='channel'),
  Regular(100, -50, 50, name='pv_z'),
  storage=Weight()) # Sum: WeightedSum(value=49850, variance=80903.1), 'pv_rho': Hist(
  StrCategory(['2mu2e'], name='channel'),
  Regular(100, -0.5, 0.5, name='pv_rho'),
  storage=Weight()) # Sum: WeightedSum(value=49850, variance=80903.1), 'electron_n': Hist(
  StrCategory(['2mu2e'], name='channel'),
  Integer(0, 10, name='electron_n'),
  storage=Weight()) # Sum: WeightedSum(value=1913.48, va

In [13]:
# test that ouput is accumulatable
from coffea.processor import accumulate
double_out = accumulate([output["out"], output["out"]])

ValueError: 'PV filter' is not in list

In [7]:
out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table()
out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table(fraction=True)

cut name        individual cut N    all cut N
------------  ------------------  -----------
No selection             11518.1      11518.1
PV filter                11518.1      11518.1
Cosmic veto              11518.1      11518.1
>=2 LJs                   1914.4       1914.4
2mu2e                     1913.5       1913.5
cut name        individual %    marginal %    cumulative %
------------  --------------  ------------  --------------
No selection           100.0         100.0           100.0
PV filter              100.0         100.0           100.0
Cosmic veto            100.0         100.0           100.0
>=2 LJs                 16.6          16.6            16.6
2mu2e                   16.6         100.0            16.6


In [ ]:
double_out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table()
double_out["2Mu2E_100GeV_1p2GeV_9p6mm"]["cutflow"]["2mu2e"].print_table(fraction=True)